In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.text)

Data Scraping and putting it in a list called table_data

In [2]:
mytable=soup.find("table", {"class": "wikitable sortable"})
data = mytable.tbody.find_all("tr") 
table_data=[]
for i in range(1,len(data)):
    for td in data[i].find_all("td"):
        table_data.append(td.text.strip())


print(table_data)

['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned', 'M3A', 'North York', 'Parkwoods', 'M4A', 'North York', 'Victoria Village', 'M5A', 'Downtown Toronto', 'Harbourfront', 'M6A', 'North York', 'Lawrence Heights', 'M6A', 'North York', 'Lawrence Manor', 'M7A', 'Downtown Toronto', "Queen's Park", 'M8A', 'Not assigned', 'Not assigned', 'M9A', "Queen's Park", 'Not assigned', 'M1B', 'Scarborough', 'Rouge', 'M1B', 'Scarborough', 'Malvern', 'M2B', 'Not assigned', 'Not assigned', 'M3B', 'North York', 'Don Mills North', 'M4B', 'East York', 'Woodbine Gardens', 'M4B', 'East York', 'Parkview Hill', 'M5B', 'Downtown Toronto', 'Ryerson', 'M5B', 'Downtown Toronto', 'Garden District', 'M6B', 'North York', 'Glencairn', 'M7B', 'Not assigned', 'Not assigned', 'M8B', 'Not assigned', 'Not assigned', 'M9B', 'Etobicoke', 'Cloverdale', 'M9B', 'Etobicoke', 'Islington', 'M9B', 'Etobicoke', 'Martin Grove', 'M9B', 'Etobicoke', 'Princess Gardens', 'M9B', 'Etobicoke', 'West Deane Park', 'M1C

Seperating The Postal Code,Borough and Neighbours in to 3 different list

In [3]:
Postalcode=[]
Borough=[]
Neighbourhood=[]
for i in range(0,len(table_data),3):
    Postalcode.append(table_data[i])
    Borough.append(table_data[i+1])
    Neighbourhood.append(table_data[i+2])
               

Forming the DataFrame as asked in the assignment

In [4]:
d={'PostalCode':Postalcode,'Borough':Borough,'Neighborhood':Neighbourhood}
df=pd.DataFrame(data=d)
print(df.tail())

    PostalCode       Borough           Neighborhood
282        M8Z     Etobicoke              Mimico NW
283        M8Z     Etobicoke     The Queensway West
284        M8Z     Etobicoke  Royal York South West
285        M8Z     Etobicoke         South of Bloor
286        M9Z  Not assigned           Not assigned


Ignoring the Not assigned Borough

In [5]:
df.drop(df[df['Borough']=='Not assigned'].index,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Combining the rows with the same PostalCode 

In [6]:
df = df.groupby('PostalCode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Assigning Not assigned Neighborhood to the corresponding Borough values

In [7]:
import numpy
df['Neighborhood']=np.where(df['Neighborhood']=='Not assigned',df['Borough'],df['Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Describing the data

In [8]:
df.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,11,102
top,M4V,North York,Queen's Park
freq,1,24,2


In [9]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


GeoCoder library not responding so using the external dataset

In [10]:
'''import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google(Postalcode[0]) 
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]'''

'import geocoder # import geocoder\n\n# initialize your variable to None\nlat_lng_coords = None\n\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n  g = geocoder.google(Postalcode[0]) \n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]'

Using the external latitude-longitude dataset and forming the Toronto dataframe with latitiude and longitude.

In [12]:
df_coord=pd.read_csv('Geospatial_Coordinates.csv')
df
merge_df = pd.merge(left=df,right=df_coord, left_on='PostalCode', right_on='Postal Code')
merge_df.drop('Postal Code',axis=1)
print(merge_df.describe())
merge_df.head()

         Latitude   Longitude
count  103.000000  103.000000
mean    43.704608  -79.397153
std      0.052463    0.097146
min     43.602414  -79.615819
25%     43.660567  -79.464763
50%     43.696948  -79.388790
75%     43.745320  -79.340923
max     43.836125  -79.160497


,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
